In [62]:
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib as plt
import scipy as sp
import seaborn as sbs

In [47]:
import datetime

In [55]:
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv')

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
airlines_incl = ["AA", "AS", "B6", "DL", "F9", "NK", "UA", "WN"]
flights_sub = flights[flights.AIRLINE.isin(airlines_incl)]

In [57]:
flights_sub['DATE'] = pd.to_datetime(flights_sub[['YEAR','MONTH', 'DAY']])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
#_________________________________________________________
# Function that converts the 'HHMM' string to datetime.time
def format_hour(nums):
    if pd.isnull(nums):
        return np.nan
    else:
        if nums == 2400: nums = 0
        nums = "{0:04d}".format(int(nums))
        hour = datetime.time(int(nums[0:2]), int(nums[2:4]))
        return hour

In [59]:
flights_sub['DEPARTURE_TIME'] = flights_sub['DEPARTURE_TIME'].apply(format_hour)
flights_sub['SCHEDULED_ARRIVAL'] = flights_sub['SCHEDULED_ARRIVAL'].apply(format_hour)
flights_sub['ARRIVAL_TIME'] = flights_sub['ARRIVAL_TIME'].apply(format_hour)

flights_sub.loc[:5, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME',
             'ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY
0,5,04:30:00,23:54:00,04:08:00,-11.0,-22.0
1,10,07:50:00,00:02:00,07:41:00,-8.0,-9.0
3,20,08:05:00,00:15:00,07:56:00,-5.0,-9.0
4,25,03:20:00,00:24:00,02:59:00,-1.0,-21.0
5,25,06:02:00,00:20:00,06:10:00,-5.0,8.0


In [60]:
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       'MONTH','DAY', 'DIVERTED']
flights_sub.drop(variables_to_remove, axis = 1, inplace = True)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
from sklearn.linear_model import Lasso

In [71]:
from sklearn.cross_validation import train_test_split
import random

In [ ]:
random.seed(1)
train_flights,test_flights = train_test_split(flights_sub,test_size=0.2)